# 微众银行外呼预测


## Step 1: 检视源数据集

In [5]:
import numpy as np
import pandas as pd
import os

#### 读入数据

* 数据已经事先导出为input

* 数据的第一列就是索引，而且两个input的索引不重复

In [ ]:
data_F = pd.read_csv('../input1.csv', index_col=0)
data_S = pd.read_csv('../input2.csv', index_col=0)

#### 检视源数据

In [ ]:
data_F.head()

In [ ]:
data_S.head()

这时候大概心里可以有数，哪些地方需要人为的处理一下，以做到源数据更加好被process。

## Step 2: 合并数据

这么做主要是为了用DF进行数据预处理的时候更加方便。等所有的需要的预处理进行完之后，我们再把他们分隔开。

首先，CallAnswer作为我们的训练目标

我们先看一下*CallAnswer*均不均衡：

In [ ]:
#合并数据
all_df = pd.concat((data_F, data_S), axis=0)

In [ ]:
y_train = all_df.CallAnswer

In [ ]:
data_F.value_counts 

可见，CallAnswer本身并不均衡。为了我们分类器的学习更加准确，我们会首先处理CallAnswer的均衡问题

第一次时就miss掉，导致自己的结果总是达不到一定标准。

In [ ]:
all_df.tail()

此刻，我们可以看到all_df就是我们合在一起的DF

In [ ]:
all_df.shape

而*y_train*则是*CallAnswer*那一列

In [ ]:
y_train.head()

## Step 3: 变量转化

类似『特征工程』。就是把不方便处理数据类型处理一下。

#### 正确化变量属性

首先，我们注意到，*MSSubClass* 的值其实应该是一个category，

但是Pandas是不会懂这些事儿的。使用DF的时候，这类数字符号会被默认记成数字。

这种东西就很有误导性，我们需要把它变回成*string*

In [ ]:
all_df['MSSubClass'].dtypes

In [ ]:
all_df['MSSubClass'] = all_df['MSSubClass'].astype(str)

变成*str*以后，做个统计，就很清楚了

In [ ]:
all_df.shape

In [ ]:
all_df['MSSubClass'].isnull().any()

In [ ]:
all_df['MSSubClass'].value_counts()

#### 把category的变量转变成numerical表达形式

当我们用numerical来表达categorical的时候，要注意，数字本身有大小的含义，所以乱用数字会给之后的模型学习带来麻烦。于是我们可以用One-Hot的方法来表达category。

pandas自带的get_dummies方法，可以帮你一键做到One-Hot。

In [ ]:
pd.get_dummies(all_df['MSSubClass'], prefix='MSSubClass').head()

此刻*MSSubClass*被我们分成了12个column，每一个代表一个category。是就是1，不是就是0。

同理，我们把所有的category数据，都给One-Hot了

In [ ]:
all_dummy_df = pd.get_dummies(all_df)
all_dummy_df.head()

In [ ]:
all_dummy_df.shape

#### 处理好numerical变量

就算是numerical的变量，也还会有一些小问题。

比如，有一些数据是缺失的：

In [ ]:
all_dummy_df.isnull().sum().sort_values(ascending=False).head(10)

可以看到，缺失最多的column是LotFrontage

处理这些缺失的信息时，询问过项目经理，

在这里，我们用平均值来填满这些空缺。

In [ ]:
mean_cols = all_dummy_df.mean()
mean_cols.head(10)

In [ ]:
all_dummy_df = all_dummy_df.fillna(mean_cols)

In [ ]:
all_dummy_df.tail()

看看是不是没有空缺了？

In [ ]:
all_dummy_df.isnull().sum().sum()

#### 标准化numerical数据

这一步并不是必要，但是得看你想要用的分类器是什么。一般来说，不要让数据间的差距太大。

这里，我们当然不需要把One-Hot的那些0/1数据给标准化。我们的目标应该是那些本来就是numerical的数据：

先来看看 哪些是numerical的：

In [ ]:
numeric_cols = all_df.columns[all_df.dtypes != 'object']
numeric_cols

In [ ]:
numeric_cols.shape

计算标准分布：(X-X')/s

让我们的数据点更平滑，更便于计算。

In [ ]:
numeric_col_means = all_dummy_df.loc[:, numeric_cols].mean()
numeric_col_std = all_dummy_df.loc[:, numeric_cols].std()
all_dummy_df.loc[:, numeric_cols] = (all_dummy_df.loc[:, numeric_cols] - numeric_col_means) / numeric_col_std

In [ ]:
all_dummy_df.tail()

## Step 4: 建立模型

#### 把数据集分成 训练/测试集

In [ ]:
from sklearn.model_selection import train_test_split
train_X,test_X,train_y,test_y = train_test_split(all_dummy_df.values,y_train.values)

In [ ]:
train_X.tail()

In [ ]:
train_X.shape, test_X.shape

#### 过采样过程

由于数据量较少，最好采用过采样

In [ ]:
 from imblearn.over_sampling import SMOTE # 导入过采样处理库SMOTE

# 使用SMOTE方法进行过采样处理

smote = SMOTE() # 建立SMOTE模型对象

X_new, Y_new = smote.fit_sample(X_train.values, Y_test.values) # 输入数据并作过采样处理



#### logistic 

用logistic Regression模型来跑一遍看看。（对于不均衡的数据，这种模型可能效果很差）

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [ ]:
Cs = [0.6,0.7,0.8,0.9,1.0,1.1,1.2]
test_scores = []
for C in Cs:
    clf = LogisticRegression(penalty=l2,C=C)
    test_score = cross_val_score(clf, X_train, y_train, cv=3, scoring='precision')
    test_scores.append(np.mean(test_score))

存下所有的CV值，看看哪个alpha值更好

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(Cs, test_scores)

可见，大概C=1.1 的时候，可以把score达到0.96左右。

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
max_features = [.1, .3, .5, .7, .9, .99]
test_scores = []
for max_feat in max_features:
    clf = RandomForestRegressor(n_estimators=200, max_features=max_feat)
    test_score = cross_val_score(clf, X_train, y_train, cv=3, scoring='precision')
    test_scores.append(np.mean(test_score))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(Cs, test_scores)

可见，大概max_features=0.99 的时候，可以把score达到0.98左右,最终采用随机森林。

#### test分数

In [ ]:
from sklearn.metrics import precision_score

In [ ]:
precision_score?

In [ ]:
clf = RandomForestRegressor(n_estimators=200, max_features=0.99)
clf.fit(X_train,y_train)
y_predict = clf.predict(X_test)
score = precision_score(y_test,y_predict)

score达到0.95